<a href="https://colab.research.google.com/github/kodenshacho/SimpleVolumeRendering/blob/master/glfragment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#include <GL/glew.h>
#include <GLFW/glfw3.h>
#include <cuda_runtime.h>
#include <cuda_gl_interop.h>
#include <iostream>

const int width = 800;
const int height = 600;

// CUDA カーネル関数
__global__ void kernel(float* ptr, int width, int height) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    int idx = y * width + x;

    if (x < width && y < height) {
        // 単純な色データを生成
        ptr[4 * idx + 0] = (float)x / width; // R
        ptr[4 * idx + 1] = (float)y / height; // G
        ptr[4 * idx + 2] = 0.0f; // B
        ptr[4 * idx + 3] = 1.0f; // A
    }
}

int main() {
    // GLFW を初期化
    if (!glfwInit()) {
        std::cerr << "GLFW 初期化に失敗しました" << std::endl;
        return -1;
    }

    // ウィンドウを作成
    GLFWwindow* window = glfwCreateWindow(width, height, "CUDA OpenGL Interop", NULL, NULL);
    if (!window) {
        std::cerr << "ウィンドウ作成に失敗しました" << std::endl;
        glfwTerminate();
        return -1;
    }

    glfwMakeContextCurrent(window);

    // GLEW を初期化
    if (glewInit() != GLEW_OK) {
        std::cerr << "GLEW 初期化に失敗しました" << std::endl;
        return -1;
    }

    // OpenGL バッファオブジェクトを作成
    GLuint vbo;
    glGenBuffers(1, &vbo);
    glBindBuffer(GL_ARRAY_BUFFER, vbo);
    glBufferData(GL_ARRAY_BUFFER, width * height * 4 * sizeof(float), NULL, GL_DYNAMIC_DRAW);

    // CUDA-OpenGL リソースを登録
    cudaGraphicsResource* cuda_vbo_resource;
    cudaGraphicsGLRegisterBuffer(&cuda_vbo_resource, vbo, cudaGraphicsMapFlagsNone);

    // ページロックメモリを確保
    float* host_ptr;
    cudaMallocHost((void**)&host_ptr, width * height * 4 * sizeof(float));

    // CUDA カーネル起動のための設定
    dim3 block(16, 16);
    dim3 grid((width + block.x - 1) / block.x, (height + block.y - 1) / block.y);

    while (!glfwWindowShouldClose(window)) {
        // CUDA リソースをマッピング
        cudaGraphicsMapResources(1, &cuda_vbo_resource, 0);
        float* d_ptr;
        size_t num_bytes;
        cudaGraphicsResourceGetMappedPointer((void**)&d_ptr, &num_bytes, cuda_vbo_resource);

        // CUDA カーネルを起動
        kernel<<<grid, block>>>(d_ptr, width, height);
        cudaDeviceSynchronize();

        // リソースをアンマップ
        cudaGraphicsUnmapResources(1, &cuda_vbo_resource, 0);

        // OpenGL レンダリング
        glClear(GL_COLOR_BUFFER_BIT);
        glBindBuffer(GL_ARRAY_BUFFER, vbo);
        glVertexPointer(4, GL_FLOAT, 0, 0);
        glEnableClientState(GL_VERTEX_ARRAY);
        glDrawArrays(GL_POINTS, 0, width * height);
        glDisableClientState(GL_VERTEX_ARRAY);

        // バッファをスワップ
        glfwSwapBuffers(window);
        glfwPollEvents();
    }

    // メモリを解放
    cudaFreeHost(host_ptr);
    cudaGraphicsUnregisterResource(cuda_vbo_resource);
    glDeleteBuffers(1, &vbo);

    // GLFW を終了
    glfwDestroyWindow(window);
    glfwTerminate();

    return 0;
}